In [112]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv

In [194]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv')
weibo_o = pd.read_csv('./data/Weibo.csv')
wechat_o = pd.read_csv('./data/Wechat.csv')
news_forum_o = pd.read_csv('./data/News_forum.csv')
dianping = pd.read_csv('./data/dianping.csv',encoding = 'utf-8')

In [207]:
dianping_pos = dianping[dianping['rate']>3]
dianping_pos['SENTIMENT'] = 'Positive'
dianping_neg = dianping[dianping['rate']<3]
dianping_neg['SENTIMENT'] = 'Negative'
dianping_neu = dianping[dianping['rate']==3]
dianping_neu['SENTIMENT'] = 'Neutral'
dianping = pd.concat([dianping_pos,dianping_neg,dianping_neu])

/home/gary/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/gary/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/gary/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:/

In [227]:
dianping = dianping[['content','SENTIMENT']].dropna(axis=0, how = 'any')
dianping.columns = ['CONTENT','SENTIMENT']
dianping['CONTENT'] = dianping['CONTENT'].str.replace('\n', '').encode('utf-8')
dianping = dianping.reset_index(drop=True)

KeyError: "['content'] not in index"

In [ ]:
for i in range(len(dianping)):
    dianping['CONTENT'][i] = HanziConv.toSimplified(dianping['CONTENT'][i]).encode('utf-8')

# Sentiment data

In [213]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [214]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [215]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [218]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [219]:
length = []
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')
    length.append(len(data['CONTENT'][i]))
    if data['SENTIMENT'][i] == 'Irrelevant':
        data['SENTIMENT'][i] = 'Neutral'
length = np.array(length)

In [221]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state = 100)
train = pd.concat([train,dianping])

In [225]:
testing_folder = './data/processed/testing/'
if not os.path.exists(testing_folder):
    os.makedirs(testing_folder)
for the_file in os.listdir(testing_folder):
    file_path = os.path.join(testing_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
train_folder = './data/processed/training/'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)
for the_file in os.listdir(train_folder):
    file_path = os.path.join(train_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [226]:
for x in train['SENTIMENT'].unique():
    np.savetxt(train_folder+ x.replace('/',' ') +'.utf8',\
                   train[train['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')
for x in test['SENTIMENT'].unique():
    np.savetxt(testing_folder+ x.replace('/',' ') +'.utf8',\
                   test[test['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

UnicodeEncodeError: 'ascii' codec can't encode characters in position 0-182: ordinal not in range(128)

In [88]:
train['SENTIMENT'].value_counts()

Neutral     11928
Positive      360
Negative      210
Name: SENTIMENT, dtype: int64

In [89]:
test['SENTIMENT'].value_counts()

Neutral     2943
Positive     105
Negative      77
Name: SENTIMENT, dtype: int64

In [1]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model')

In [2]:
len(w2vModel.wv.vocab)

3577

In [68]:
from gensim.models import KeyedVectors
w2vModel = KeyedVectors.load_word2vec_format('runs/wiki.zh.vec')

In [53]:
vocab_obj = w2vModel.wv.vocab
word = u'河北'
print word
print vocab_obj[word].index

河北
3038


In [114]:
dianping_json = pd.read_table('./data/reviews.txt')

In [189]:
x = ''
for i in range(len(dianping_json)):
    x += ('{"'+dianping_json.loc[i][0].split('^ {"')[1]+'\n')

In [190]:
dianping = pd.read_json(x, lines=True)

In [192]:
dianping.to_csv('./data/dianping.csv',encoding = 'utf-8')

3293878